## Python Crawler to scrap linkedin recent posts/activity of a user

In [88]:
### Importing required libraries
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import re
from collections import Counter
import os

In [89]:
### browser params for selenium
firefox_options = Options()
firefox_options.add_argument("--incognito")
firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe' ## give firefox exe path here


In [90]:
### running the webdriver
driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present

C:\Users\Yusra Shahid\AppData\Local\Temp;C\conda_tmp\ipykernel_2228\501550139.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present


In [91]:
import time
url = "https://linkedin.com/"
driver.get(url)
time.sleep(2)

In [92]:
### sign in to linkedin
signInButton = driver.find_element(By.XPATH,"/html/body/main/section[1]/div/form[1]/div[2]/button")
signInButton.click()


In [93]:
USERNAME = input("Enter the username: ")
PASSWORD = input("Enter the password: ")
print(USERNAME)


Enter the username: yusra_shahid95@hotmail.com
Enter the password: 26June1995!
yusra_shahid95@hotmail.com


In [94]:
email = driver.find_element(By.XPATH,'//*[@id="session_key"]')
email.send_keys(USERNAME)

password = driver.find_element(By.XPATH,'//*[@id="session_password"]')
password.send_keys(PASSWORD)

In [95]:
login = driver.find_element(By.XPATH,'/html/body/main/section[1]/div/form[1]/div[2]/button')
login.click()

In [96]:
### in case there is a phone verification setup page this will skip it
try:
    skip = driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/section/div[2]/div/article/footer/div/div/button[2]')
    skip.click()
except:
    print("skip button not found")

skip button not found


In [97]:
### goto profile and then recent activity link
own_profile = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/div/div[1]/div[1]/a/div[2]')
##profile = driver.find_element(By.CSS_SELECTOR,'div.t-16:nth-child(2)')
own_profile.click()

In [98]:
activity = driver.find_elements(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]/div[3]/div/div/a')
for my_href in activity:
    activity_link = my_href.get_attribute("href")

In [99]:
## function to scrape profiles
def get_recent_activity(profile):
    prof_link = profile + "/recent-activity/"
    driver.get(prof_link)
    print("Scrapping activity of {}".format(profile))
    #### getting posts that are gathered in 20 seconds of scroll
    start=time.time()
    n =20
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        end=time.time()
        if round(end-start)>n:
            break
        #posts = driver.find_elements(By.CLASS_NAME,"scaffold-finite-scroll__content")
    posts_source = driver.page_source 
    linkedin_soup = bs4(posts_source.encode("utf-8"), "html")
    linkedin_soup.prettify()
    containers = linkedin_soup.findAll("div",{"class":"ember-view occludable-update"})
    conn_names = Counter()
    p_text,urls =[],[]
    for container in containers:

        try:
            ## get poster's name
            name_box = container.find("div",{"class":"update-components-actor"})
            name = name_box.find("a")['href'].split("?")[0]
            #name  = name.text.strip()
            conn_names.update([name]) 
            text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
            text = text_box.find("span",{"dir":"ltr"})
            post_text = text.text.strip()
            p_text.append(post_text)
            #print(post_text)
            if "https" in post_text:
                post_url = re.findall("(?P<url>https?://[^\s]+)", post_text)
            else:
                post_url = ""
            #print(post_url)
            urls.append(post_url)
        except:
            #print(text_box)
            pass
    print("total number of posts are: ",len(p_text))
    print("total number of urls are: ",len(urls)-urls.count(""))
    print("interacted with whom",conn_names)
    return p_text,urls,conn_names


In [100]:
##print(activity_link)
#driver.get(activity_link)
#time.sleep(5)
own_profile = activity_link.replace("/recent-activity/","")
p_text,urls,conn_names = get_recent_activity(own_profile)

Scrapping activity of https://www.linkedin.com/in/yusra-shahid
total number of posts are:  22
total number of urls are:  5
interacted with whom Counter({'https://www.linkedin.com/in/yusra-shahid': 4, 'https://www.linkedin.com/in/rahulneelmani': 1, 'https://www.linkedin.com/in/ali-bhayani': 1, 'https://www.linkedin.com/in/ido-lev-ran-3b67205': 1, 'https://www.linkedin.com/in/hesham-farag-b3218052': 1, 'https://www.linkedin.com/in/ashishpatel2604/profile-video/view/urn%3Ali%3Afsd%5Fstory%3Aurn%3Ali%3Astory%3AACoAAATOnMIBYFNuqVOB33Dkrrhcd4bGpGp6FIM': 1, 'https://www.linkedin.com/in/airockstar': 1, 'https://www.linkedin.com/in/maksym-nechepurenko-23541810': 1, 'https://www.linkedin.com/in/inshatazeen': 1, 'https://www.linkedin.com/in/zahrakhan12': 1, 'https://www.linkedin.com/in/annaradulovski': 1, 'https://www.linkedin.com/in/milyasyousuf': 1, 'https://www.linkedin.com/in/eczachly': 1, 'https://www.linkedin.com/in/emanuelefabbiani': 1, 'https://www.linkedin.com/company/ucfcrcv/': 1, 'http

In [101]:
del conn_names[own_profile]
conn_names

Counter({'https://www.linkedin.com/in/rahulneelmani': 1,
         'https://www.linkedin.com/in/ali-bhayani': 1,
         'https://www.linkedin.com/in/ido-lev-ran-3b67205': 1,
         'https://www.linkedin.com/in/hesham-farag-b3218052': 1,
         'https://www.linkedin.com/in/ashishpatel2604/profile-video/view/urn%3Ali%3Afsd%5Fstory%3Aurn%3Ali%3Astory%3AACoAAATOnMIBYFNuqVOB33Dkrrhcd4bGpGp6FIM': 1,
         'https://www.linkedin.com/in/airockstar': 1,
         'https://www.linkedin.com/in/maksym-nechepurenko-23541810': 1,
         'https://www.linkedin.com/in/inshatazeen': 1,
         'https://www.linkedin.com/in/zahrakhan12': 1,
         'https://www.linkedin.com/in/annaradulovski': 1,
         'https://www.linkedin.com/in/milyasyousuf': 1,
         'https://www.linkedin.com/in/eczachly': 1,
         'https://www.linkedin.com/in/emanuelefabbiani': 1,
         'https://www.linkedin.com/company/ucfcrcv/': 1,
         'https://www.linkedin.com/in/laillab': 1,
         'https://www.linked

In [ ]:
for profile,count in conn_names.items():
    if count >=0:
        get_recent_activity(profile)

Scrapping activity of https://www.linkedin.com/in/rahulneelmani
total number of posts are:  18
total number of urls are:  0
interacted with whom Counter({'https://www.linkedin.com/in/rahulneelmani': 12, 'https://www.linkedin.com/in/pooja-agrawalla': 2, 'https://www.linkedin.com/in/drrajiyer': 2, 'https://www.linkedin.com/in/abhijeet-ajoynil-074b0015': 1, 'https://www.linkedin.com/in/sarajitjha': 1})
Scrapping activity of https://www.linkedin.com/in/ali-bhayani
total number of posts are:  20
total number of urls are:  6
interacted with whom Counter({'https://www.linkedin.com/company/cloud-shuttle/': 11, 'https://www.linkedin.com/in/ali-bhayani': 2, 'https://www.linkedin.com/in/peterhanssens': 2, 'https://www.linkedin.com/in/iyanaarmstrong': 1, 'https://www.linkedin.com/in/mohammedalimcs': 1, 'https://www.linkedin.com/in/haoyu-martin': 1, 'https://www.linkedin.com/in/muhammad-raza-b-0a37751a0': 1, 'https://www.linkedin.com/in/rashid-mohammed-1a19a6179': 1, 'https://www.linkedin.com/in/mu